In [48]:
import os
from rdkit import Chem
from rdkit.Chem import AllChem

In [58]:
# setting
path = os.getcwd()
pdb = '7ugw'
lig1 = 'alatrofloxacin'
lig2 = 'quarfloxin'

# preparing protein
os.system(f'PATH="$PATH":{path}/ADFRsuite/bin')
os.system(f'ADFRsuite/bin/prepare_receptor -r data/{pdb}.pdb -o {pdb}.pdbqt')

0

In [ ]:
def cas2smiles(cas):
    import pubchempy as pcp
    # this is used for the ligand with CAS number but without SMILES
    com = pcp.get_compounds(cas, 'name')
    return com[0].isomeric_smiles

def smiles2pdb(smiles, name):
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, randomSeed=1)
    mol.SetProp('SMILES', smiles)
    mol = Chem.MolToPDBBlock(mol)
    print(mol, file=open(f"data/{name}.pdb", "w"))
    return

def smiles2pdb_optimized(smiles, name):
    # this is used for applying MMFF94 force field to optimize the ligand conformation
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMultipleConfs(mol, numConfs=numConfs,
                                useRandomCoords=True,
                                maxAttempts=maxAttempts,
                                enforceChirality=True,
                                useExpTorsionAnglePrefs=True,
                                useBasicKnowledge=False)
    for conf in mol.GetConformers():
        new_mol = Chem.Mol(mol)
        new_mol.RemoveAllConformers()
        new_mol.AddConformer(conf)
        AllChem.MMFFOptimizeMolecule(new_mol)
    mol = new_mol.GetConformers()[0]
    mol.SetProp('SMILES', smiles)
    mol = Chem.MolToPDBBlock(mol)
    print(mol, file=open(f"data/{name}.pdb", "w"))
    return

# load ligand
df = pd.read_csv('data/ligands.csv')
ligand1, ligand2 = df['SMILES'].values.tolist()
smiles2pdb(ligand1, lig1)
smiles2pdb(ligand2, lig2)

# preparing ligand
os.system(f'ADFRsuite/bin/prepare_ligand -l data/{lig1}.pdb -o {lig1}.pdbqt')
os.system(f'ADFRsuite/bin/prepare_ligand -l data/{lig2}.pdb -o {lig2}.pdbqt')

In [ ]:
# get grid center
os.system(f'pythonsh prepare_gpf.py -r data/{pdb}.pdbqt -l {lig1}.pdbqt -y')
with open('7ugw.gpf', 'r') as f:
    for i in f.readlines():
        if i.startswith('gridcenter'):
            coord = [float(j) for j in i.split(' ')[1:4]]
f.close()
print(coord)

os.system(f'pythonsh prepare_gpf.py -r data/{pdb}.pdbqt -l {lig2}.pdbqt -y')
with open('7ugw.gpf', 'r') as f:
    for i in f.readlines():
        if i.startswith('gridcenter'):
            coord = [float(j) for j in i.split(' ')[1:4]]
f.close()
print(coord)